<a href="https://colab.research.google.com/github/ArkS0001/Chat-PDFs/blob/main/PDF_and_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import login

# Login to Hugging Face
login(token="hf_gvaeNZYabgNlundlofemeebWFqSkYYA")

In [6]:
pip install transformers gradio PyPDF2 langchain faiss-cpu sentence-transformers


In [10]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [14]:
import os
import shutil
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from transformers import pipeline
import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Load environment variables (if needed)
load_dotenv()

# Set up the text generation pipeline using Meta LLaMA 3.3
generator = pipeline(
    "text-generation",
    model="meta-llama/Meta-Llama-3-8B",
    device_map="auto",
    torch_dtype="auto",
)

# Function to extract text from uploaded PDF files
def get_pdf_text(pdf_files):
    text = ""
    for pdf_file in pdf_files:
        pdf_reader = PdfReader(pdf_file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# Split text into manageable chunks
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    return text_splitter.split_text(text)

# Create or load the FAISS vector store
def create_vector_store(text_chunks):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")
    return vector_store

# Load the vector store and retrieve relevant context
def get_relevant_context(question, top_k=5):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    vector_store = FAISS.load_local("faiss_index", embeddings)
    docs = vector_store.similarity_search(question, k=top_k)
    return " ".join([doc.page_content for doc in docs])

# Function to generate a response using LLaMA 3.3
def generate_response(context, question):
    input_text = f"Context:\n{context}\n\nQuestion: {question}"
    response = generator(input_text, max_length=500, temperature=0.7, num_return_sequences=1)
    return response[0]["generated_text"]

# Gradio interface for PDF-based Q&A
def chatbot_interface(pdf_files, question):
    # Extract text from PDF files
    text = get_pdf_text(pdf_files)

    # Split the text into chunks and create/load the FAISS index
    text_chunks = get_text_chunks(text)
    if not os.path.exists("faiss_index"):
        create_vector_store(text_chunks)

    # Retrieve relevant context
    context = get_relevant_context(question)

    # Generate response
    response = generate_response(context, question)
    return response

# Gradio app definition
with gr.Blocks() as app:
    gr.Markdown("# 📄 Chat with Your PDFs using LLaMA 3.3")

    with gr.Row():
        pdf_input = gr.File(label="Upload PDF(s)", file_types=[".pdf"], file_count="multiple")
        question_input = gr.Textbox(label="Ask a Question")
        submit_button = gr.Button("Submit")

    output = gr.Textbox(label="Response", interactive=False)

    submit_button.click(chatbot_interface, inputs=[pdf_input, question_input], outputs=output)

# Run the app
if __name__ == "__main__":
    app.launch()


model-00003-of-00004.safetensors:  51%|#####     | 2.51G/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cpu


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://02df38713d78435912.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
